### Шаг №1 - Инициализация класса



Инициализация класса
Приступим к реализации. И первое, что мы сделаем — это создадим класс MyLineReg.

Данный класс при инициализации должен принимать на вход два параметра:

n_iter — количество шагов градиентного спуска.
По-умолчанию: 100

learning_rate — коэффициент скорости обучения градиентного спуска.
По-умолчанию: 0.1

Все переданные (или дефолтные) параметры должны быть сохранены внутри класса.

При обращении к экземпляру класса (или при передачи его в функцию print) необходимо распечатать строку по следующему шаблону (строго в таком виде):

MyLineReg class: n_iter=<n_iter>, learning_rate=<learning_rate>

Примечания:

Параметры должны распечатываться точно в таком же порядке, как приведены выше.
В следующих уроках будут добавляться новые параметры. Если вы захардкодите вывод параметров, то в последующем их придется вручную добавлять в распечатку. Либо можете сделать динамический вывод всех имеющихся параметров.
(Здесь и далее) вам необходимо написать только класс. Создавать экземпляр класса не нужно.

In [ ]:
class MyLineReg():
  def __init__(self, n_iter=100, learning_rate=0.1):
    self.n_iter = n_iter
    self.learning_rate = learning_rate

  def __str__(self):
    params = ", ".join(f"{key}={value}" for key, value in self.__dict__.items())
    return f"{__class__.__name__} class: {params}"

### Шаг №2 - Метод fit.



Пора научить нашу модель чему-то полезному :) Что для это нужно сделать:

1. В инициализатор класса добавить новый параметр — weights — который будет хранить веса модели. По умолчанию он ничего не содержит.
2. Вам необходимо реализовать метод fit в Вашем классе. Данный метод должен делать следующее:
  1. На вход принимать три атрибута:
  - X — все фичи в виде датафрейма пандаса.
  Примечание: даже если фича будет всего одна это все равно будет датафрейм, а не серия.
  - y — целевая переменная в виде пандасовской серии.
  - verbose — указывает на какой итерации выводить лог. Например,значение 10 означает, что на каждой 10 итерации градиентного спуска будет печататься лог. Значение по умолчанию: False (т.е. ничего не выводится).
  2. Дополнить переданную матрицу фичей единичным столбцом слева.
  3. Определить сколько фичей передано и создать вектор весов, состоящий из одних единиц соответствующей длинны: т.е. количество фичей + 1.
  4. Дальше в цикле (до n_iter):
  - Предсказать целевую переменную
  - Посчитать ошибку (MSE)
  - Вычислить градиент
  - Сделать шаг размером learning rate в противоположную от градиента сторону
  - Сохранить обновленные веса внутри класса
  5. В процессе обучения необходимо выводить лог, в котором указывать номер итерации и значение функций потерь:
  start | loss: 42027.65
  100 | loss: 1222.87
  200 | loss: 232.17
  300 | loss: 202.4
  где start - значении функции потерь до начала обучения. Далее выводится каждое i-ое значение итерации переданное в параметре verbose. Если verbose = False, то лог не выводится вовсе.
  З.Ы. Данный вывод никак проверяться не будет. Он в основном нужен для отладки. Поэтому можете модифицировать его внешний вид под свои нужды.
  6. Метод ничего не возвращает.

3. Необходимо реализовать метод get_coef, который будет возвращать значения весов в виде вектора NumPy, начиная со второго значения. Первое значение нам не нужно, потому что оно соответствует фиктивной фиче (единичке). Все же остальные могут использоваться для оценки важности фичей.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import numpy as np
import pandas as pd

class MyLineReg:
    def __init__(self, n_iter=100, learning_rate=0.1, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights

    def __str__(self):
        params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != 'weights')
        return f"{__class__.__name__} class: {params}"

    __repr__ = __str__

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        # Добавляем столбец единичек для фиктивной фичи (свободного члена)
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])

        # Инициализируем веса
        self.weights = np.ones(X.shape[1])

        # Вычисляем начальное значение функции потерь
        y_pred = X.dot(self.weights)
        loss = np.mean((y_pred - y) ** 2)

        # Лог перед началом обучения
        if verbose:
            print(f"start | loss: {loss:.2f}")

        # Градиентный спуск
        for i in range(1, self.n_iter + 1):
            # Предсказания
            y_pred = X.dot(self.weights)

            # Ошибка (Mean Squared Error)
            loss = np.mean((y_pred - y) ** 2)

            # Градиент
            gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)

            # Обновление весов
            self.weights -= self.learning_rate * gradient

            # Логирование каждые 'verbose' итераций
            if verbose and i % verbose == 0:
                print(f"{i} | loss: {loss:.2f}")

    def get_coef(self):
        # Возвращаем все веса, начиная с первого (пропускаем фиктивный признак)
        return self.weights[1:]

In [ ]:
# создадим набор тестовых данных для проверки класса

import numpy as np
import pandas as pd

# Задание параметров
np.random.seed(42)  # Для воспроизводимости

# Количество примеров
n_samples = 1000

# Генерация данных признаков X (два признака x1 и x2)
X = np.random.rand(n_samples, 2) * 10  # Признаки от 0 до 10

# Коэффициенты, которые мы знаем (истинные веса)
true_weights = np.array([2, 7])  # для x1 и x2
bias = 7  # Свободный член (константа)

# Генерация целевой переменной y с шумом
y = X.dot(true_weights) + bias + np.random.randn(n_samples) * 0.5  # Немного шума

# Превращаем в DataFrame для удобства работы
X_df = pd.DataFrame(X, columns=['x1', 'x2'])
y_series = pd.Series(y)

# Выводим примеры данных
print('Признаки')
print(X_df.head(10))  # Признаки
print('Целевая переменная')
print(y_series.head(10))  # Целевая переменная

Признаки
         x1        x2
0  3.745401  9.507143
1  7.319939  5.986585
2  1.560186  1.559945
3  0.580836  8.661761
4  6.011150  7.080726
5  0.205845  9.699099
6  8.324426  2.123391
7  1.818250  1.834045
8  3.042422  5.247564
9  4.319450  2.912291
Целевая переменная
0    80.601813
1    63.132533
2    20.926750
3    68.977685
4    69.044173
5    74.903790
6    39.258935
7    23.339253
8    49.807111
9    35.651334
dtype: float64


In [ ]:
# Создаем объект класса MyLineReg
model = MyLineReg(n_iter=1000, learning_rate=0.01)

# Обучаем модель
model.fit(X_df, y_series, verbose=100)

# Получаем коэффициенты
print("Оцененные коэффициенты:", model.get_coef())

# Истинные коэффициенты (должны быть близки к 2 и 7)
print("Истинные коэффициенты: [2, 7]")

start | loss: 2015.46
100 | loss: 2.60
200 | loss: 1.57
300 | loss: 0.98
400 | loss: 0.66
500 | loss: 0.47
600 | loss: 0.37
700 | loss: 0.31
800 | loss: 0.28
900 | loss: 0.26
1000 | loss: 0.25
Оцененные коэффициенты: [2.02176664 7.01914741]
Истинные коэффициенты: [2, 7]


### Шаг 3: Добавление метода `predict` для предсказаний



Теперь добавим третий шаг, чтобы наша модель могла выдавать предсказания.

#### Задача:

Добавьте в класс `MyLineReg` метод `predict`. Этот метод должен выполнять следующие действия:

1. На вход принимать матрицу фичей в виде датафрейма Pandas.
2. Дополнять матрицу фичей единичным вектором (первый столбец).
3. Возвращать вектор предсказаний.

#### Напоминание:

Предсказание вычисляется следующим образом:

$$
\hat{y} = XW
$$

Где:
- $X$ — матрица фичей (с единичным столбцом для свободного члена).
- $W$ — вектор весов, обученный моделью.

In [ ]:
class Zati4ka():
  # добавляем в наш класс метод предикт
  def predict(self, X: pd.DataFrame):
    # дополняем матрицу фичей единичным вектором
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # возвращаем предсказанные значения
    return X.dot(self.weights)

In [ ]:
# Теперь наш класс будет выглядеть так:
class MyLineReg():
  # инициализация класса
  def __init__(self, n_iter=100, learning_rate=0.1, weights=None):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights

  # метод str, для вывода информации
  def __str__(self):
    params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != 'weights')
    return f"{__class__.__name__} class: {params}"

  # реализация метода fit
  # метод принимает:
  # X - все фичи в виде датафрейма пандаса;
  # y — целевая переменная в виде пандасовской серии
  # verbose — указывает на какой итерации выводить лог
  def fit(self, X: pd.DataFrame, y:pd.Series, verbose=False):

    # Дополним переданную матрицу фичей - единичным столбцом слева
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])

    # Инициализация весов
    self.weights = np.ones(X.shape[1])

    # Вычисляем начальное значение функции потерь
    # .dot - выполняет метричное умножение X * self.weights
    y_pred = X.dot(self.weights)
    loss = np.mean((y_pred - y) ** 2)

    # Лог перед началом обучения
    if verbose:
      print(f"start | loss: {loss:.2f}")

    # реализация градиентного спуска
    for i in range(1, self.n_iter + 1):
      # Предсказания
      y_pred = X.dot(self.weights)

      # Функция потерь (Mean Squared Error)
      loss = np.mean((y - y_pred) ** 2)

      # вычисляем градиент функции потерь
      gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)

      # Обновление весов
      self.weights -= self.learning_rate * gradient

      # Логирование каждые 'verbose' итераций
      if verbose and i % verbose == 0:
        print(f"{i} | loss: {loss:.2f}")

  def get_coef(self):
    #возвращаем все веса, начиная с первого (пропускаем фиктивный признак)

    return self.weights[1:]

    # добавляем в наш класс метод предикт
  def predict(self, X: pd.DataFrame):
    # дополняем матрицу фичей единичным вектором
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # возвращаем предсказанные значения
    return X.dot(self.weights)

#### Опробуем метод предикт на новых данных

In [ ]:
# Создаем объект класса MyLineReg
model = MyLineReg(n_iter=1000, learning_rate=0.01)

# Обучаем модель
model.fit(X_df, y_series, verbose=100)

start | loss: 2015.46
100 | loss: 2.60
200 | loss: 1.57
300 | loss: 0.98
400 | loss: 0.66
500 | loss: 0.47
600 | loss: 0.37
700 | loss: 0.31
800 | loss: 0.28
900 | loss: 0.26
1000 | loss: 0.25


In [ ]:
# создадим набор новых тестовых данных для проверки класса

import numpy as np
import pandas as pd

# Задание параметров
np.random.seed(42)  # Для воспроизводимости

# Количество примеров
n_samples = 1000

# Генерация данных признаков X (два признака x1 и x2)
Test_data = np.random.rand(n_samples, 2) * 10  # Признаки от 0 до 10

# Коэффициенты, которые мы знаем (истинные веса)
true_weights = np.array([14, 72])  # для x1 и x2
bias = 7  # Свободный член (константа)

# Генерация целевой переменной y с шумом
y_test = Test_data.dot(true_weights) + bias + np.random.randn(n_samples) * 0.5  # Немного шума

# Превращаем в DataFrame для удобства работы
Test_df = pd.DataFrame(Test_data, columns=['x1', 'x2'])
y_series_test = pd.Series(y_test)

# Выводим примеры данных
print('Признаки')
print(X_df.head(10))  # Признаки
print('Целевая переменная')
print(y_series.head(10))  # Целевая переменная

Признаки
         x1        x2
0  3.745401  9.507143
1  7.319939  5.986585
2  1.560186  1.559945
3  0.580836  8.661761
4  6.011150  7.080726
5  0.205845  9.699099
6  8.324426  2.123391
7  1.818250  1.834045
8  3.042422  5.247564
9  4.319450  2.912291
Целевая переменная
0    80.601813
1    63.132533
2    20.926750
3    68.977685
4    69.044173
5    74.903790
6    39.258935
7    23.339253
8    49.807111
9    35.651334
dtype: float64


In [ ]:
y_pred = model.predict(Test_df)

In [ ]:
print("Предсказания:", y_pred)

Предсказания: [81.06413081 63.57969585 20.8635832  68.73226061 68.61356581 75.25533772
 38.49420785 23.30927448 49.74426068 35.93448798 28.92128247 38.38172592
 71.09309398 46.89157621 21.99742812 31.0122438  74.67863103 83.02519287
 19.77409428 51.48831248 43.98429061 71.28156011 58.49510813 49.56617662
 30.78817782 80.77021908 88.563492   83.55561883 22.30520294 30.50957076
 33.66430136 48.55594543 50.53224329 65.91631258 77.5380539  36.32089879
 64.10984907 72.22085876 27.55035489 22.14012565 67.9599165  17.91100523
 35.8721986  66.26179809 57.84259799 59.2413846  61.53793723 57.0071631
 47.33793732 14.84673877 52.06577471 48.81263589 42.6068303  68.08999823
 16.78886338 23.93422307 82.27924271 80.73481432 36.10376646 62.66245212
 85.98228999 20.91376003 41.34740739 83.71406312 42.75042694 30.78891609
 32.88086961 48.50928912 66.5944378  82.3223239  43.89117389 37.93206154
 15.10772435 54.36751503 27.35915133 41.93798398 44.04461196 43.67753344
 73.80798081 62.67882058 58.57796005 5

### Шаг №4 - Метрики (реализация)





Теперь добавим возможность использовать различные метрики для оценки качества модели.

#### Задача:

1. **Добавьте в класс `MyLineReg` параметр `metric`**, который будет принимать одно из следующих значений:
   - `mae` — средняя абсолютная ошибка (Mean Absolute Error).
   - `mse` — среднеквадратичная ошибка (Mean Squared Error).
   - `rmse` — корень из среднеквадратичной ошибки (Root Mean Squared Error).
   - `mape` — средняя абсолютная процентная ошибка (Mean Absolute Percentage Error).
   - `r2` — коэффициент детерминации \(R^2\).
2. По умолчанию: `None`.
3. При обучении добавьте в вывод расчёт метрики:

Пример вывода:

```
start | loss: 42027.65 | <metric_name>: 234.65
100 | loss: 1222.87 | <metric_name>: 114.35
200 | loss: 232.17 | <metric_name>: 58.2
300 | loss: 202.4 | <metric_name>: 46.01
```

4. Если метрика не задана, то ничего дополнительно выводить не нужно.

5. **Добавьте метод `get_best_score`**, который возвращает последнее значение метрики (после завершения обучения модели).

### Примечания:

- Градиентный спуск **всё ещё выполняется по среднеквадратичной ошибке (MSE)**, даже если отслеживаются другие метрики.
- Почему добавляем метрику `MSE`, если функция потерь — это уже `MSE`?
  - Во-первых, функцией потерь может быть не только `MSE`.
  - Во-вторых, мы добавим регуляризацию, которая повлияет на функцию потерь, но **не должна влиять на метрику**.

In [ ]:
import numpy as np
import pandas as pd

class MyLineReg:
  def __init__(self, n_iter=100, learning_rate=0.1, metric=None, weights=None):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.metric = metric
    self.weights = weights
    self.best_score = None

  def __str__(self):
    params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != "weights")
    return f"{__class__.__name__} class: {params}"

  def compute_metric(self, y_true, y_pred):
    if self.metric == 'mae':
      return np.mean(np.abs(y_true - y_pred))
    elif self.metric == 'mse':
      return np.mean((y_true - y_pred) ** 2)

    elif self.metric == 'rmse':
      return np.sqrt(np.mean((y_true - y_pred) ** 2))
    elif self.metric == 'mape':
      return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    elif self.metric == 'r2':
      ss_res = np.sum((y_true - y_pred) ** 2)
      ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
      return 1 - (ss_res / ss_tot)
    return None

  def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
    # Добавляем единичный столбец для смещения (биаса)
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])

    # Инициализируем веса единицами
    self.weights = np.ones(X.shape[1])

    for i in range(1, self.n_iter + 1):
      y_pred = X.dot(self.weights)
      loss = np.mean((y_pred - y) ** 2)

      # Градиент
      gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)

      # Обновляем веса
      self.weights -= self.learning_rate * gradient

      # Логирование на каждой итерации
      if verbose and i % verbose == 0:
        metric_value = self.compute_metric(y, y_pred)
        if self.metric:
          print(f"{i} | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}")
        else:
          print(f"{i} | loss: {loss:.2f}")

    # Вычисляем метрику на последнем шаге после завершения всех итераций
    y_pred_final = X.dot(self.weights)  # Предсказания с обновлёнными весами
    self.best_score = round(self.compute_metric(y, y_pred_final), 10)

  def get_best_score(self):
    return self.best_score

  # добавляем в наш класс метод предикт
  def predict(self, X: pd.DataFrame):
    # дополняем матрицу фичей единичным вектором
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # возвращаем предсказанные значения
    return X.dot(self.weights)


#### Общий принцип работы кода:
1. Инициализация параметров модели.
2. Подготовка данных (добавление единичного столбца).
3. Итерации градиентного спуска:
   - Предсказание текущих значений на основе текущих весов.
   - Вычисление ошибки (функции потерь).
   - Обновление весов с помощью градиентного спуска.
4. Подсчет и вывод выбранной метрики (если она указана).
5. Возвращение результата.

Теперь разберем это по шагам.

#### 1. **Инициализация параметров модели**
Когда ты создаешь объект класса `MyLineReg`, запускается конструктор:

```python
class MyLineReg:
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.best_score = None
```

- **`n_iter`**: Сколько раз ты будешь обновлять веса (число итераций).
- **`learning_rate`**: Насколько сильно изменяются веса на каждой итерации (скорость обучения).
- **`metric`**: Какая метрика будет использоваться для отслеживания качества модели.
- **`weights`**: Веса модели (они пока не заданы, это будет сделано позже).
- **`best_score`**: Сохраняется финальный результат выбранной метрики после завершения обучения.

_Этот шаг выполняется, когда ты создаешь объект модели, например:_
```python
model = MyLineReg(n_iter=300, learning_rate=0.05, metric='mae')
```

#### 2. **Подготовка данных**
Данные, с которыми ты работаешь, состоят из признаков $X$ и целевой переменной $y$. Для работы модели нам нужно:
- Добавить единичный столбец к признакам $X$, чтобы учесть смещение (bias).
- Инициализировать веса.

```python
X = np.hstack([np.ones((X.shape[0], 1)), X.values])
self.weights = np.ones(X.shape[1])
```

##### Подробное описание:
- **`X.shape[0]`**: Число строк в матрице признаков (количество примеров).
- **`np.hstack`**: Добавляет столбец единиц к $X$. Это нужно для учёта смещения. Теперь $X$ выглядит так:

$$
X =
\begin{pmatrix}
1 & x_{11} & x_{12} & ... & x_{1n} \\
1 & x_{21} & x_{22} & ... & x_{2n} \\
\vdots & \vdots & \vdots & & \vdots \\
1 & x_{m1} & x_{m2} & ... & x_{mn}
\end{pmatrix}
$$

Каждый пример данных теперь имеет дополнительную фиктивную (единичную) переменную, которая соответствует смещению $b$ в модели.

- **Инициализация весов**: Веса (коэффициенты регрессии) инициализируются единицами, по одному весу на каждый признак. Если $X$ имеет $n$ признаков, веса будут:

$$
W = [1, 1, 1, ... , 1]
$$

#### 3. **Итерации градиентного спуска**
Теперь начинается ключевая часть — итерации градиентного спуска. Это цикл, в котором на каждом шаге происходят следующие действия:

##### 3.1. **Предсказание**
```python
y_pred = X.dot(self.weights)
```
На каждой итерации ты используешь текущие веса для предсказания $y_{pred}$, используя формулу:

$$
y_{pred} = X \cdot W
$$

- **`X.dot(self.weights)`**: Умножает матрицу признаков $X$ на текущий вектор весов $W$. Это предсказание значений целевой переменной $y_{pred}$.

##### Пример:
Допустим, у тебя есть 2 признака и 3 примера:

$$
X =
\begin{pmatrix}
1 & 2 & 3 \\
1 & 4 & 5 \\
1 & 6 & 7
\end{pmatrix}
$$
Текущие веса:

$$
W = [1, 0.5, -0.5]
$$
Предсказания:

$$
y_{pred} = X \cdot W = [1 \cdot 1 + 2 \cdot 0.5 + 3 \cdot (-0.5), 1 \cdot 1 + 4 \cdot 0.5 + 5 \cdot (-0.5), 1 \cdot 1 + 6 \cdot 0.5 + 7 \cdot (-0.5)] = [0.5, 0.5, 0.5]
$$

##### 3.2. **Вычисление функции потерь (MSE)**
```python
loss = np.mean((y_pred - y) ** 2)
```
Здесь вычисляется среднеквадратичная ошибка (MSE):

$$
\text{MSE} = \frac{1}{m} \sum_{i=1}^m (y_{pred_i} - y_i)^2
$$
Где $m$ — количество примеров в данных, $y_i$ — реальные значения, а $y_{pred_i}$ — предсказанные.

##### 3.3. **Вычисление градиента**
```python
gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)
```

- **Градиент** — это производная функции потерь по весам. Он показывает направление, в котором нужно двигаться, чтобы уменьшить ошибку.

$$
\text{Градиент} = \frac{2}{m} X^T \cdot (y_{pred} - y)
$$

- **`X.T.dot(y_pred - y)`**: Сначала вычисляется разница между предсказанными и реальными значениями ($y_{pred} - y$), затем умножается на транспонированную матрицу признаков $X^T$. Это даёт вектор, который указывает, как сильно нужно изменить каждый вес, чтобы уменьшить ошибку.

##### 3.4. **Обновление весов**
```python
self.weights -= self.learning_rate * gradient
```
Теперь мы обновляем веса:

$$
W = W - \alpha \cdot \text{Градиент}
$$

Где:
- $\alpha$ — скорость обучения (learning rate),
- **Градиент** — направление, в котором нужно обновить веса.

Это шаг градиентного спуска. Веса изменяются, чтобы на следующем шаге уменьшить функцию потерь.

#### 4. **Подсчет метрики и логирование**
```python
metric_value = self.compute_metric(y, y_pred)
```

В конце каждой итерации (или через каждые несколько итераций, если используется `verbose`), ты вычисляешь выбранную метрику (например, MAE, MSE и т.д.). Она позволяет отслеживать качество модели в процессе обучения.

#### 5. **Конец обучения**
После завершения всех итераций модель сохраняет последнее значение метрики:

```python
self.best_score = round(self.compute_metric(y, y_pred_final), 10)
```

Теперь модель завершила обучение, и ты можешь использовать метод `get_best_score`, чтобы получить финальную метрику.

#### Визуализация работы кода в голове

1. **Инициализация**:
   - Модель начинает с нулевых весов (всех единиц).
   - Данные $X$ дополняются единичным столбцом для учёта смещения.

2. **Итерации обучения**:
   - Каждый шаг итерации можно представить как «движение» в пространстве весов, где модель постепенно находит такие значения весов, которые минимизируют ошибку.
   - На каждом шаге модель делает предсказания, вычисляет ошибку, затем обновляет веса в направлении уменьшения этой ошибки.

3. **Финальная метрика**:
   - Когда обучение завершено, модель готова делать финальные предсказания с улучшенными весами. Метрика на последнем шаге оценивает, насколько хорошо модель справляется с задачей.

### Шаг №5 - Регуляризация (реализация)




Добавьте в класс `MyLineReg` три параметра:

- `reg` – принимает одно из трех значений: `'l1'`, `'l2'`, `'elasticnet'`.  
  По умолчанию: `None`.
- `l1_coef` – принимает значения от 0.0 до 1.0.  
  По умолчанию: `0`.
- `l2_coef` – принимает значения от 0.0 до 1.0.  
  По умолчанию: `0`.

Добавьте регуляризацию к вычислению лосса.  
Добавьте регуляризацию к вычислению градиента.

#### Примечания:

- Для вычисления регуляризации L1 вам нужно задать `reg="l1"` и указать только `l1_coef`.
- Для вычисления L2 вам нужно задать `reg="l2"` и указать только `l2_coef`.
- Для вычисления Elasticnet вам нужно задать `reg="elasticnet"` и указать оба параметра `l1_coef` и `l2_coef`.

### Проверка

**Входные данные**: три вида регуляризации и одна модель без регуляризации (`None`).  
**Выходные данные**: коэффициенты обученной линейной регрессии (их сумма).

In [ ]:
print(MyLineReg())

MyLineReg class: n_iter=100, learning_rate=0.1, metric=None, best_score=None, reg=None, l1_coef=0.0, l2_coef=0.0


In [ ]:
import numpy as np
import pandas as pd

class MyLineReg:
    def __init__(self,
                 n_iter=100,
                 learning_rate=0.1,
                 metric=None,
                 weights=None,
                 reg=None,
                 l1_coef=0.0,
                 l2_coef=0.0):

      self.n_iter = n_iter
      self.learning_rate = learning_rate
      self.metric = metric
      self.weights = weights
      self.best_score = None
      self.reg = reg  # Регуляризация ('l1', 'l2', 'elasticnet', или None)
      self.l1_coef = l1_coef  # Коэффициент для L1 регуляризации
      self.l2_coef = l2_coef  # Коэффициент для L2 регуляризации

    def __str__(self):
        params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != "weights")
        return f"{__class__.__name__} class: {params}"

    def compute_metric(self, y_true, y_pred):
        if self.metric == 'mae':
            return np.mean(np.abs(y_true - y_pred))
        elif self.metric == 'mse':
            return np.mean((y_true - y_pred) ** 2)
        elif self.metric == 'rmse':
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif self.metric == 'mape':
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        elif self.metric == 'r2':
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            return 1 - (ss_res / ss_tot)
        return None

    def compute_loss(self, y, y_pred):
        # Основной лосс — MSE
        loss = np.mean((y_pred - y) ** 2)

        # Добавляем регуляризацию к лоссу
        if self.reg == 'l1':
            loss += self.l1_coef * np.sum(np.abs(self.weights))  # L1 регуляризация
        elif self.reg == 'l2':
            loss += self.l2_coef * np.sum(self.weights ** 2)  # L2 регуляризация
        elif self.reg == 'elasticnet':
            loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)  # ElasticNet

        return loss

    def compute_gradient(self, X, y, y_pred):
        # Обычный градиент
        gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)

        # Добавляем регуляризацию к градиенту
        if self.reg == 'l1':
            gradient += self.l1_coef * np.sign(self.weights)  # Для L1 добавляем знак весов
        elif self.reg == 'l2':
            gradient += 2 * self.l2_coef * self.weights  # Для L2 градиент добавляется как 2 * lambda * w
        elif self.reg == 'elasticnet':
            gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights  # Для ElasticNet

        return gradient

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        # Добавляем единичный столбец для смещения (биаса)
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])

        # Инициализируем веса единицами
        self.weights = np.ones(X.shape[1])

        for i in range(1, self.n_iter + 1):
            y_pred = X.dot(self.weights)
            loss = self.compute_loss(y, y_pred)

            # Градиент
            gradient = self.compute_gradient(X, y, y_pred)

            # Обновляем веса
            self.weights -= self.learning_rate * gradient

            # Логирование на каждой итерации
            if verbose and i % verbose == 0:
                metric_value = self.compute_metric(y, y_pred)
                if self.metric:
                    print(f"{i} | loss: {loss:.6f} | {self.metric}: {metric_value:.6f}")
                else:
                    print(f"{i} | loss: {loss:.6f}")

        # Вычисляем метрику на последнем шаге после завершения всех итераций
        y_pred_final = X.dot(self.weights)  # Предсказания с обновлёнными весами
        self.best_score = round(self.compute_metric(y, y_pred_final), 10)

    def get_best_score(self):
        return self.best_score

    # Метод для получения коэффициентов (весов)
    def get_coef(self):
        return self.weights

    # Метод для предсказания на новых данных
    def predict(self, X: pd.DataFrame):
        # Дополняем матрицу фичей единичным вектором
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])
        # Возвращаем предсказанные значения
        return X.dot(self.weights)


### Шаг №6 - Скорость обучения

параметр learning rate теперь будет определяться callable

Возьмите код из предыдущего шага и модифицируйте в нем параметр learning_rate следующим:

Если на вход пришло число, то работаем как и раньше.
Если на вход пришла lambda-функция, то вычисляем learning_rate на каждом шаге на основе переданной лямбда-функции.
Можете дополнительно для контроля вывести значение learning_rate в лог тренировки.

Примечания:

Т.к. у нас теперь результат зависит от нумерации шагов, то формализуем их нумерацию: они должна считаться от 1 до n_iter (включительно).

In [ ]:
# Ключевая часть кода, в которой были произведены изменения

            # Вычисление шага обучения
            if callable(self.learning_rate):
                lr = self.learning_rate(i)  # Используем лямбда-функцию для вычисления шага
            else:
                lr = self.learning_rate

            # Градиент
            gradient = self.compute_gradient(X, y, y_pred)

            # Обновляем веса
            self.weights -= lr * gradient

In [ ]:
import numpy as np
import pandas as pd

class MyLineReg:
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None, weights=None, reg=None, l1_coef=0.0, l2_coef=0.0):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.best_score = None
        self.reg = reg  # Регуляризация (None, 'l1', 'l2', 'elasticnet')
        self.l1_coef = l1_coef  # Коэффициент для L1 регуляризации
        self.l2_coef = l2_coef  # Коэффициент для L2 регуляризации

    def __str__(self):
        params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != "weights")
        return f"{__class__.__name__} class: {params}"

    def compute_metric(self, y_true, y_pred):
        if self.metric == 'mae':
            return np.mean(np.abs(y_true - y_pred))
        elif self.metric == 'mse':
            return np.mean((y_true - y_pred) ** 2)
        elif self.metric == 'rmse':
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif self.metric == 'mape':
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        elif self.metric == 'r2':
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            return 1 - (ss_res / ss_tot)
        return None  # Возвращает None, если метрика не задана или не поддерживается

    def compute_loss(self, y, y_pred):
        # Обычный MSE Loss
        loss = np.mean((y_pred - y) ** 2)
        # Добавляем регуляризацию к лоссу
        if self.reg == 'l1':  # L1 регуляризация
            loss += self.l1_coef * np.sum(np.abs(self.weights))  # Не применяем регуляризацию к bias (весам смещения)
        elif self.reg == 'l2':  # L2 регуляризация
            loss += self.l2_coef * np.sum(self.weights ** 2)
        elif self.reg == 'elasticnet':  # ElasticNet регуляризация
            loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)
        return loss

    def compute_gradient(self, X, y, y_pred):
        # Обычный градиент
        gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)
        # Добавляем регуляризацию к градиенту
        if self.reg == 'l1':  # L1 регуляризация (только на весах, не на смещении)
            gradient += self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':  # L2 регуляризация
            gradient += 2 * self.l2_coef * self.weights
        elif self.reg == 'elasticnet':  # ElasticNet регуляризация
            gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
        return gradient

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        # Добавляем единичный столбец для смещения (биаса)
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])
        # Инициализируем веса единицами
        self.weights = np.ones(X.shape[1])
        for i in range(1, self.n_iter + 1):
            y_pred = X.dot(self.weights)
            loss = self.compute_loss(y, y_pred)
            # Вычисление шага обучения
            if callable(self.learning_rate):
                lr = self.learning_rate(i)  # Используем лямбда-функцию для вычисления шага
            else:
                lr = self.learning_rate
            # Градиент
            gradient = self.compute_gradient(X, y, y_pred)
            # Обновляем веса
            self.weights -= lr * gradient
            # Логирование на каждой итерации
            if verbose and i % verbose == 0:
                metric_value = self.compute_metric(y, y_pred)
                if self.metric:
                    print(f"{i} | loss: {loss:.6f} | learning_rate: {lr:.6f}")
                else:
                    print(f"{self.metric}: {metric_value:.6f}")
        # Вычисляем метрику на последнем шаге после завершения всех итераций
        y_pred_final = X.dot(self.weights)  # Предсказания с обновлёнными весами
        # Вычисляем лучшую метрику, если она задана, иначе присваиваем значение None
        final_metric_value = self.compute_metric(y, y_pred_final)
        if final_metric_value is not None:
            self.best_score = round(final_metric_value, 10)
        else:
            self.best_score = None

    def get_best_score(self):
        return self.best_score

    # Новый метод для получения коэффициентов (весов)
    def get_coef(self):
        # Возвращаем все веса, кроме первого элемента (bias)
        return self.weights[1:]

    # Метод для предсказания на новых данных
    def predict(self, X: pd.DataFrame):
        # Дополняем матрицу фичей единичным вектором
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])
        # Возвращаем предсказанные значения
        return X.dot(self.weights)

### Шаг №7 - Стохастический градиентный спуск (реализация)

Добавьте в класс MyLineReg два новых параметра:

- sgd_sample – кол-во образцов, которое будет использоваться на каждой итерации обучения. Может принимать либо целые числа, либо дробные от 0.0 до 1.0.
По-умолчанию: None
- random_state – для воспроизводимости результата зафиксируем сид (об этом далее).
По-умолчанию: 42

Внесем изменение в алгоритм обучения:

- В начале обучения фиксируем сид (см. ниже).
- В начале каждого шага формируется новый мини-пакет, состоящий из случайно выбранных элементов обучающего набора. Кол-во отобранных элементов определяется параметром sgd_sample:
  - Если задано целое число, то из исходного датасета берется ровно столько примеров сколько указано.
  - Если задано дробное число, то рассматриваем его как долю от количества строк в исходном датасете (округленное до целого числа).
- Расчет градиента (и последующее изменение весов) делаем на основе мини-пакета.
- Все остальные параметры, если они заданы (например, регуляризация), также должны учитываться при обучении.
- Ошибку и метрику необходимо считать на всем датасете, а не на мини-пакете.
- Если sgd_sample = None, то обучение выполняется как раньше (на всех данных).

#### Случайная генерация

Т.к. у нас формальная проверка кода, то у всех должны получиться одинаковые случайные подвыборки. Поэтому и способ у всех будет одинаковый.

В начале обучения посредством модуля random фиксируем сид:

```random.seed(<random_state>)```

В начале каждой итерации сформируем порядковые номера строк, которые стоит отобрать.

```sample_rows_idx = random.sample(range(X.shape[0]), <sgd_sample>)```

В этом случае при каждом запуске будут генерироваться одни и те же номера строк. Что позволит нам добиться воспроизводимости.

З.Ы. Модуль random уже импортирован.

З.Ы.2. При отборе строк не стоит полагаться на индекс пандаса – он может быть не последовательным.

In [ ]:
import numpy as np
import pandas as pd
import random

class MyLineReg:
    def __init__(self,
                 n_iter=100,
                 learning_rate=0.1,
                 metric=None,
                 weights=None,
                 reg=None,
                 l1_coef=0.0,
                 l2_coef=0.0,
                 sgd_sample=None,
                 random_state=42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.best_score = None
        self.reg = reg  # Регуляризация (None, 'l1', 'l2', 'elasticnet')
        self.l1_coef = l1_coef  # Коэффициент для L1 регуляризации
        self.l2_coef = l2_coef  # Коэффициент для L2 регуляризации
        self.sgd_sample = sgd_sample # Количество выборок для SGD
        self.random_state = random_state  # Для воспроизводимости

    def __str__(self):
        params = ', '.join(f"{key}={value}" for key, value in self.__dict__.items() if key != "weights")
        return f"{__class__.__name__} class: {params}"

    def compute_metric(self, y_true, y_pred):
        if self.metric == 'mae':
            return np.mean(np.abs(y_true - y_pred))
        elif self.metric == 'mse':
            return np.mean((y_true - y_pred) ** 2)
        elif self.metric == 'rmse':
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif self.metric == 'mape':
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        elif self.metric == 'r2':
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            return 1 - (ss_res / ss_tot)
        return None  # Возвращает None, если метрика не задана или не поддерживается

    def compute_loss(self, y, y_pred):
        # Обычный MSE Loss
        loss = np.mean((y_pred - y) ** 2)
        # Добавляем регуляризацию к лоссу
        if self.reg == 'l1':  # L1 регуляризация
            loss += self.l1_coef * np.sum(np.abs(self.weights))  # Не применяем регуляризацию к bias (весам смещения)
        elif self.reg == 'l2':  # L2 регуляризация
            loss += self.l2_coef * np.sum(self.weights ** 2)
        elif self.reg == 'elasticnet':  # ElasticNet регуляризация
            loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)
        return loss

    def compute_gradient(self, X, y, y_pred):
        # Обычный градиент
        gradient = (2 / X.shape[0]) * X.T.dot(y_pred - y)
        # Добавляем регуляризацию к градиенту
        if self.reg == 'l1':  # L1 регуляризация (только на весах, не на смещении)
            gradient += self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':  # L2 регуляризация
            gradient += 2 * self.l2_coef * self.weights
        elif self.reg == 'elasticnet':  # ElasticNet регуляризация
            gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
        return gradient

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        # фиксируем сид для воспроизводимости
        random.seed(self.random_state)
        # Добавляем единичный столбец для смещения (биаса)
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])
        # Инициализируем веса единицами
        self.weights = np.ones(X.shape[1])
        # Определяем кол-во примеров для мини-батча
        if self.sgd_sample is not None:
          if isinstance(self.sgd_sample, float) and 0 < self.sgd_sample <= 1:
            batch_size = int(self.sgd_sample * X.shape[0])  # Интерпретируем как долю от данных
          elif isinstance(selg.sgd_sample, int) and self.sgd_sample > 0:
            batch_size = self.sgd_sample  # Определенное количество строк
          else:
            raise VelueError("sgd_sample должен быть целым числом или дробным значением от 0 до 1.")
        else:
          batch_size = X.shape[0]  # Если не задано, то берем все данные
        for i in range(1, self.n_iter + 1):
            # Формируем мини батч
            if batch_size < X.shape[0]:
              sample_rows_idx = random.sample(range(X.shape[0]), batch_size)
              X_batch = X[sample_rows_idx]
              y_batch = y.values[sample_rows_idx]
            else:
              X_batch = X
              y_batch = y
            # Предсказания для мини-батча
            y_pred_batch = X_batch.dot(self.weights)
            # Градиент
            gradient = self.compute_gradient(X_batch, y_batch, y_pred_batch)
            # Вычисление шага обучения
            if callable(self.learning_rate):
              lr = self.learning_rate(i)  # Используем лямбда-функцию для вычисления шага
            else:
              lr = self.learning_rate
            # Обновляем веса
            self.weights -= lr*gradient
            # Полное предсказание всех данных и вычисление метрики/лосса
            # на всех данных
            y_pred = X.dot(self.weights)
            loss = self.compute_loss(y, y_pred)
            # Логирование на каждой итерации
            if verbose and i % verbose == 0:
                metric_value = self.compute_metric(y, y_pred)
                if self.metric:
                    print(f"{i} | loss: {loss:.6f} | learning_rate: {lr:.6f}")
                else:
                    print(f"{self.metric}: {metric_value:.6f}")
        # Вычисляем метрику на последнем шаге после завершения всех итераций
        y_pred_final = X.dot(self.weights)  # Предсказания с обновлёнными весами
        # Вычисляем лучшую метрику, если она задана, иначе присваиваем значение None
        final_metric_value = self.compute_metric(y, y_pred_final)
        if final_metric_value is not None:
            self.best_score = round(final_metric_value, 10)
        else:
            self.best_score = None

    def get_best_score(self):
        return self.best_score

    # Новый метод для получения коэффициентов (весов)
    def get_coef(self):
        # Возвращаем все веса, кроме первого элемента (bias)
        return self.weights[1:]

    # Метод для предсказания на новых данных
    def predict(self, X: pd.DataFrame):
        # Дополняем матрицу фичей единичным вектором
        X = np.hstack([np.ones((X.shape[0], 1)), X.values])
        # Возвращаем предсказанные значения
        return X.dot(self.weights)

### Шаг №8 - Тестируем получившийся класс

#### Синтетические данные

In [ ]:
import numpy as np
import pandas as pd
import random

# Функция для генерации синтетических данных
def generate_data(n_samples=1000, n_features=3, noise=0.1, random_state=42):
    np.random.seed(random_state)  # Фиксируем сид для воспроизводимости
    random.seed(random_state)

    # Генерируем случайные коэффициенты для линейной модели
    true_weights = np.random.randn(n_features)
    bias = np.random.randn()  # Генерируем смещение (bias)

    # Генерируем случайные данные для признаков
    X = np.random.randn(n_samples, n_features)

    # Генерируем целевые значения с добавлением случайного шума
    y = X.dot(true_weights) + bias + noise * np.random.randn(n_samples)

    # Преобразуем данные в DataFrame для удобства
    X_df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(n_features)])
    y_series = pd.Series(y, name='target')

    return X_df, y_series, true_weights, bias

# Генерация данных
X_train, y_train, true_weights, true_bias = generate_data(n_samples=100, n_features=3, noise=0.1)

# Вывод истинных коэффициентов и смещения
print(f"True coefficients (weights): {true_weights}")
print(f"True bias: {true_bias}")


True coefficients (weights): [ 0.49671415 -0.1382643   0.64768854]
True bias: 1.5230298564080254


In [ ]:
model = MyLineReg(n_iter=1000, learning_rate=0.1, metric='mse', sgd_sample=0.1, random_state=42)
model.fit(X_train, y_train, verbose=100)
learned_weights = model.get_coef()
print(f"Learned coefficients: {learned_weights}")
print(f"True coefficients: {true_weights}")


100 | loss: 0.007748 | learning_rate: 0.100000
200 | loss: 0.008134 | learning_rate: 0.100000
300 | loss: 0.007689 | learning_rate: 0.100000
400 | loss: 0.007626 | learning_rate: 0.100000
500 | loss: 0.007683 | learning_rate: 0.100000
600 | loss: 0.007876 | learning_rate: 0.100000
700 | loss: 0.007558 | learning_rate: 0.100000
800 | loss: 0.007847 | learning_rate: 0.100000
900 | loss: 0.007830 | learning_rate: 0.100000
1000 | loss: 0.007656 | learning_rate: 0.100000
Learned coefficients: [ 0.52060662 -0.14627314  0.63750582]
True coefficients: [ 0.49671415 -0.1382643   0.64768854]


### Шаг №9 - Пояснения с примерами того, как работает линейная регрессия от ChatGPT. Для наглядности

Давай детально разберем метод `fit` и его математические выкладки.

### Математические выкладки метода `fit`

#### 1. **Постановка задачи**
Метод `fit` реализует обучение линейной модели, цель которой — найти набор весов **w** и смещение **b**, которые минимизируют функцию потерь (например, среднеквадратическую ошибку, MSE) на обучающей выборке.

**Линейная модель** имеет следующий вид:

$$
\hat{y} = X \cdot w + b
$$

где:
- $X$ — это матрица признаков, каждая строка соответствует одному объекту, каждая колонка — признаку,
- $w$ — это вектор весов (коэффициентов) для признаков,
- $b$ — смещение (bias),
- $\hat{y}$ — предсказанные значения целевой переменной.

#### 2. **Функция потерь**

Часто используется **MSE** (mean squared error, среднеквадратическая ошибка), которая вычисляется по формуле:

$
L(w, b) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$

где:
- $y_i$ — истинное значение целевой переменной,
- $\hat{y}_i$ — предсказанное значение,
- $n$ — количество обучающих примеров.

Добавляется регуляризация:
- **L1-регуляризация (Lasso):**
$$
L(w) = \lambda_1 \sum_{j=1}^{p} |w_j|
$$
- **L2-регуляризация (Ridge):**
$$
L(w) = \lambda_2 \sum_{j=1}^{p} w_j^2
$$
- **ElasticNet** (комбинация L1 и L2):
$$
L(w) = \lambda_1 \sum_{j=1}^{p} |w_j| + \lambda_2 \sum_{j=1}^{p} w_j^2
$$

#### 3. **Обновление весов**

Обучение модели основано на методе **градиентного спуска**, который итеративно обновляет веса $w$ и смещение $b$.

На каждой итерации градиент потерь по весам вычисляется как:

$$
\frac{\partial L}{\partial w_j} = \frac{2}{n} \sum_{i=1}^{n} (X_i \cdot w + b - y_i) \cdot X_{ij}
$$

$$
\frac{\partial L}{\partial b} = \frac{2}{n} \sum_{i=1}^{n} (X_i \cdot w + b - y_i)
$$

Обновление весов происходит с использованием правила:

$$
w_j = w_j - \eta \cdot \frac{\partial L}{\partial w_j}
$$

где:
- $\eta$ — это коэффициент обучения (learning rate).

#### 4. **Мини-батчи (SGD)**

Если указана опция `sgd_sample`, используется стохастический градиентный спуск (SGD). Вместо обновления весов на основе всех данных, используется случайная выборка, которая ускоряет обучение:

$$
w_j = w_j - \eta \cdot \frac{\partial L_{mini-batch}}{\partial w_j}
$$

### Пример работы на простых данных

#### 1. **Инициализация данных**

Пусть у нас есть маленькая матрица \( X \) с 3 примерами и 2 признаками, а также вектор целевой переменной \( y \).

```python
X = pd.DataFrame({
    'x1': [1, 2, 3],
    'x2': [4, 5, 6]
})
y = pd.Series([7, 8, 9])
```

Эти данные выглядят так:

$$
X = \begin{pmatrix}
1 & 4 \\
2 & 5 \\
3 & 6 \\
\end{pmatrix}, \quad y = \begin{pmatrix}
7 \\
8 \\
9 \\
\end{pmatrix}
$$

#### 2. **Подготовка и инициализация**

При вызове метода `fit`, на первом этапе к матрице \( X \) добавляется столбец с единицами, чтобы учесть смещение (bias):

$$
X' = \begin{pmatrix}
1 & 1 & 4 \\
1 & 2 & 5 \\
1 & 3 & 6 \\
\end{pmatrix}
$$
Инициализируем веса **единицами**:

$$
w = \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix}
$$

#### 3. **Первый шаг обучения**

Теперь вычислим предсказанные значения:

$$
\hat{y} = X' \cdot w = \begin{pmatrix}
1 & 1 & 4 \\
1 & 2 & 5 \\
1 & 3 & 6 \\
\end{pmatrix} \cdot \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix} = \begin{pmatrix} 6 \\ 8 \\ 10 \end{pmatrix}
$$

Вычислим ошибку (разницу между предсказанными и реальными значениями):

$$
\text{error} = \hat{y} - y = \begin{pmatrix} 6 \\ 8 \\ 10 \end{pmatrix} - \begin{pmatrix} 7 \\ 8 \\ 9 \end{pmatrix} = \begin{pmatrix} -1 \\ 0 \\ 1 \end{pmatrix}
$$

Градиент для весов (без регуляризации):

$$
\text{gradient} = \frac{2}{n} X'^T \cdot (\hat{y} - y) = \frac{2}{3} \begin{pmatrix}
1 & 1 & 1 \\
1 & 2 & 3 \\
4 & 5 & 6 \\
\end{pmatrix} \cdot \begin{pmatrix} -1 \\ 0 \\ 1 \end{pmatrix} = \begin{pmatrix} 0 \\ 0.6667 \\ 0.6667 \end{pmatrix}
$$

Теперь обновляем веса, используя шаг обучения $\eta = 0.1$:

$$
w = w - \eta \cdot \text{gradient} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix} - 0.1 \cdot \begin{pmatrix} 0 \\ 0.6667 \\ 0.6667 \end{pmatrix} = \begin{pmatrix} 1 \\ 0.93333 \\ 0.93333 \end{pmatrix}
$$

#### 4. **Последующие шаги**

Этот процесс повторяется для заданного количества итераций \( n\_iter \), пока веса не сойдутся к оптимальным значениям.

### Заключение

Метод `fit` вашего класса реализует градиентный спуск для минимизации функции потерь, учитывая возможную регуляризацию. Важные моменты:
- добавление единичного столбца для смещения (bias),
- вычисление градиентов и обновление весов,
- опция для стохастического градиентного спуска с использованием мини-батчей.

На простых данных можно видеть, как веса обновляются на основе градиента, постепенно приближаясь к оптимальным значениям.